In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import FinanceLib as fl

%matplotlib inline
%load_ext autoreload
%autoreload 2

C:\Projects\finance-venv\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [ ]:
#load stock list
fl.save_sp500_tickers()

In [4]:
# os.environ['PG_PWD']

In [5]:
#check what load in work
fl.save_stock_quote_to_db("AAPL", "2023-01-01", "2023-06-11")

[*********************100%***********************]  1 of 1 completed


0

In [7]:
#activate logger
today_dt = fl.date.today().strftime('%Y%m%d')    
log_dir = fl.LOG_DIR + '/logs/' + today_dt

dt_to = fl.date.today()
dt_from = fl.date.today() + fl.datetime.timedelta(days=-2)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
fl.logging.basicConfig(filename=log_dir + '/app.log', filemode='a+', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%H:%M:%S', level=fl.logging.DEBUG)
logger = fl.logging.getLogger()

In [8]:
fl.get_all_stock_by_period("2023-01-01", "2023-06-20", logger)

C:\Projects\FinanceProject\FinanceLib.py:131: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


BRK.B
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted
BF.B
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted
NRG
[*********************100%***********************]  1 of 1 completed
NUE
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
NVR
[*********************100%***********************]  1 of 1 completed
NXPI
[*********************100%***********************]  1 of 1 completed
ORLY
[*********************100%***********************]  1 of 1 completed
OXY
[*********************100%***********************]  1 of 1 completed
ODFL
[*********************100%***********************]  1 of 1 completed
OMC
[*********************100%***********************]  1 of 1 completed
ON
[*********************100%****

[*********************100%***********************]  1 of 1 completed
TRV
[*********************100%***********************]  1 of 1 completed
TRMB
[*********************100%***********************]  1 of 1 completed
TFC
[*********************100%***********************]  1 of 1 completed
TYL
[*********************100%***********************]  1 of 1 completed
TSN
[*********************100%***********************]  1 of 1 completed
USB
[*********************100%***********************]  1 of 1 completed
UDR
[*********************100%***********************]  1 of 1 completed
ULTA
[*********************100%***********************]  1 of 1 completed
UNP
[*********************100%***********************]  1 of 1 completed
UAL
[*********************100%***********************]  1 of 1 completed
UPS
[*********************100%***********************]  1 of 1 completed
URI
[*********************100%***********************]  1 of 1 completed
UNH
[*********************100%***********************

{'State': 3, 'CntError': 0, 'lst_error': []}